In [313]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime

## Earnings reports & some macroeconomic factors

In [314]:
AAPL_q = yf.Ticker('AAPL') \
  .get_income_stmt(freq='quarterly') \
  .T[[
      'NetIncome',
      'TotalRevenue',
      'EBITDA',
      'EBIT',
      'InterestExpense',
      'InterestIncome',
      'ResearchAndDevelopment',
      'GrossProfit',
      'OperatingIncome',
      'BasicEPS',
      'DilutedEPS'
]]
AAPL_q = AAPL_q.sort_index()
AAPL_q.insert(0, 'EndDate', AAPL_q.index)
AAPL_q.insert(0, 'StartDate', AAPL_q.index.to_series().shift(1) + pd.Timedelta(days=1))
AAPL_q.insert(0, 'Granularity', 'Quarterly')

AAPL_y = yf.Ticker('AAPL') \
  .get_income_stmt(freq='yearly') \
  .T[[
      'NetIncome',
      'TotalRevenue',
      'EBITDA',
      'EBIT',
      'InterestExpense',
      'InterestIncome',
      'ResearchAndDevelopment',
      'GrossProfit',
      'OperatingIncome',
      'BasicEPS',
      'DilutedEPS'
]]
AAPL_y = AAPL_y.sort_index()
AAPL_y.insert(0, 'EndDate', AAPL_y.index)
AAPL_y.insert(0, 'StartDate', AAPL_y.index.to_series().shift(1) + pd.Timedelta(days=1))
AAPL_y.insert(0, 'Granularity', 'Yearly')

AAPL = pd.concat([AAPL_q, AAPL_y])
AAPL.insert(0, 'Ticker', 'AAPL')
AAPL.reset_index(drop=True, inplace=True)
AAPL

,Ticker,Granularity,StartDate,EndDate,NetIncome,TotalRevenue,EBITDA,EBIT,InterestExpense,InterestIncome,ResearchAndDevelopment,GrossProfit,OperatingIncome,BasicEPS,DilutedEPS
0,AAPL,Quarterly,NaT,2023-09-30,NaN,NaN,NaN,NaN,1002000000.0,984000000.0,NaN,NaN,NaN,NaN,NaN
1,AAPL,Quarterly,2023-10-01,2023-12-31,33916000000.0,119575000000.0,43221000000.0,40373000000.0,NaN,NaN,7696000000.0,54855000000.0,40373000000.0,2.19,2.18
2,AAPL,Quarterly,2024-01-01,2024-03-31,23636000000.0,90753000000.0,30736000000.0,27900000000.0,NaN,NaN,7903000000.0,42271000000.0,27900000000.0,1.53,1.53
3,AAPL,Quarterly,2024-04-01,2024-06-30,21448000000.0,85777000000.0,28202000000.0,25352000000.0,NaN,NaN,8006000000.0,39678000000.0,25352000000.0,1.4,1.4
4,AAPL,Quarterly,2024-07-01,2024-09-30,14736000000.0,94930000000.0,32502000000.0,29591000000.0,NaN,NaN,7765000000.0,43879000000.0,29591000000.0,0.97,0.97
5,AAPL,Quarterly,2024-10-01,2024-12-31,36330000000.0,124300000000.0,45912000000.0,42832000000.0,NaN,NaN,8268000000.0,58275000000.0,42832000000.0,2.41,2.4
6,AAPL,Yearly,NaT,2020-09-30,NaN,NaN,NaN,NaN,2873000000.0,3763000000.0,NaN,NaN,NaN,NaN,NaN
7,AAPL,Yearly,2020-10-01,2021-09-30,94680000000.0,365817000000.0,123136000000.0,111852000000.0,2645000000.0,2843000000.0,21914000000.0,152836000000.0,108949000000.0,5.67,5.61
8,AAPL,Yearly,2021-10-01,2022-09-30,99803000000.0,394328000000.0,130541000000.0,119437000000.0,2931000000.0,2825000000.0,26251000000.0,170782000000.0,119437000000.0,6.15,6.11
9,AAPL,Yearly,2022-10-01,2023-09-30,96995000000.0,383285000000.0,125820000000.0,114301000000.0,3933000000.0,3750000000.0,29915000000.0,169148000000.0,114301000000.0,6.16,6.13


## P/E Ratio & Dividends

In [315]:
AAPL.columns

Index(['Ticker', 'Granularity', 'StartDate', 'EndDate', 'NetIncome',
       'TotalRevenue', 'EBITDA', 'EBIT', 'InterestExpense', 'InterestIncome',
       'ResearchAndDevelopment', 'GrossProfit', 'OperatingIncome', 'BasicEPS',
       'DilutedEPS'],
      dtype='object')

In [316]:
min_date = AAPL.StartDate.min()
max_date = AAPL.EndDate.max()

In [317]:
AAPL_all = yf.Ticker('AAPL').history(start=min_date, end=max_date)
AAPL_all.insert(0, 'Ticker', 'AAPL')
AAPL_all['NetIncome_q'] = pd.NA
AAPL_all['NetIncome_y'] = pd.NA
AAPL_all['TotalRevenue_q'] = pd.NA
AAPL_all['TotalRevenue_y'] = pd.NA
AAPL_all['EBITDA_q'] = pd.NA
AAPL_all['EBITDA_y'] = pd.NA
AAPL_all['EBIT_q'] = pd.NA
AAPL_all['EBIT_y'] = pd.NA
AAPL_all['InterestExpense_q'] = pd.NA
AAPL_all['InterestExpense_y'] = pd.NA
AAPL_all['InterestIncome_q'] = pd.NA
AAPL_all['InterestIncome_y'] = pd.NA
AAPL_all['ResearchAndDevelopment_q'] = pd.NA
AAPL_all['ResearchAndDevelopment_y'] = pd.NA
AAPL_all['GrossProfit_q'] = pd.NA
AAPL_all['GrossProfit_y'] = pd.NA
AAPL_all['OperatingIncome_q'] = pd.NA
AAPL_all['OperatingIncome_y'] = pd.NA
AAPL_all['BasicEPS_q'] = pd.NA
AAPL_all['BasicEPS_y'] = pd.NA
AAPL_all['DilutedEPS_q'] = pd.NA
AAPL_all['DilutedEPS_y'] = pd.NA

In [318]:
granularity_map = {
    'Quarterly': '_q',
    'Yearly': '_y'
}

for row in AAPL.itertuples():
    start = row.StartDate.tz_localize('America/New_York')
    end = row.EndDate.tz_localize('America/New_York')
    granularity = row.Granularity

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'NetIncome{granularity_map[granularity]}'
    ] = row.NetIncome

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'TotalRevenue{granularity_map[granularity]}'
    ] = row.TotalRevenue

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'EBITDA{granularity_map[granularity]}'
    ] = row.EBITDA

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'EBIT{granularity_map[granularity]}'
    ] = row.EBIT

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'InterestExpense{granularity_map[granularity]}'
    ] = row.InterestExpense

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'InterestIncome{granularity_map[granularity]}'
    ] = row.InterestIncome

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'ResearchAndDevelopment{granularity_map[granularity]}'
    ] = row.ResearchAndDevelopment

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'GrossProfit{granularity_map[granularity]}'
    ] = row.GrossProfit

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'OperatingIncome{granularity_map[granularity]}'
    ] = row.OperatingIncome

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'BasicEPS{granularity_map[granularity]}'
    ] = row.BasicEPS

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'DilutedEPS{granularity_map[granularity]}'
    ] = row.DilutedEPS

AAPL_all.head()

,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,NetIncome_q,NetIncome_y,...,ResearchAndDevelopment_q,ResearchAndDevelopment_y,GrossProfit_q,GrossProfit_y,OperatingIncome_q,OperatingIncome_y,BasicEPS_q,BasicEPS_y,DilutedEPS_q,DilutedEPS_y
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00-04:00,AAPL,114.710607,114.788617,112.945681,113.881775,116120400,0.0,0.0,<NA>,94680000000.0,...,<NA>,21914000000.0,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61
2020-10-02 00:00:00-04:00,AAPL,110.078890,112.497138,109.425576,110.205650,144712000,0.0,0.0,<NA>,94680000000.0,...,<NA>,21914000000.0,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61
2020-10-05 00:00:00-04:00,AAPL,111.073497,113.745265,110.722461,113.598999,106243800,0.0,0.0,<NA>,94680000000.0,...,<NA>,21914000000.0,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61
2020-10-06 00:00:00-04:00,AAPL,112.818923,113.228470,109.454835,110.342178,161498200,0.0,0.0,<NA>,94680000000.0,...,<NA>,21914000000.0,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61
2020-10-07 00:00:00-04:00,AAPL,111.765819,112.672661,111.288015,112.214363,96849000,0.0,0.0,<NA>,94680000000.0,...,<NA>,21914000000.0,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61


In [319]:
AAPL_all['PE_q'] = AAPL_all.Close / AAPL_all.BasicEPS_q
AAPL_all['PE_y'] = AAPL_all.Close / AAPL_all.BasicEPS_y

In [320]:
AAPL_all.head()

,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,NetIncome_q,NetIncome_y,...,GrossProfit_q,GrossProfit_y,OperatingIncome_q,OperatingIncome_y,BasicEPS_q,BasicEPS_y,DilutedEPS_q,DilutedEPS_y,PE_q,PE_y
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00-04:00,AAPL,114.710607,114.788617,112.945681,113.881775,116120400,0.0,0.0,<NA>,94680000000.0,...,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61,<NA>,20.084969
2020-10-02 00:00:00-04:00,AAPL,110.078890,112.497138,109.425576,110.205650,144712000,0.0,0.0,<NA>,94680000000.0,...,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61,<NA>,19.436623
2020-10-05 00:00:00-04:00,AAPL,111.073497,113.745265,110.722461,113.598999,106243800,0.0,0.0,<NA>,94680000000.0,...,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61,<NA>,20.035097
2020-10-06 00:00:00-04:00,AAPL,112.818923,113.228470,109.454835,110.342178,161498200,0.0,0.0,<NA>,94680000000.0,...,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61,<NA>,19.460702
2020-10-07 00:00:00-04:00,AAPL,111.765819,112.672661,111.288015,112.214363,96849000,0.0,0.0,<NA>,94680000000.0,...,<NA>,152836000000.0,<NA>,108949000000.0,<NA>,5.67,<NA>,5.61,<NA>,19.790893


## More economic factors

In [321]:
min_date = min_date - pd.Timedelta(weeks=6)
max_date = max_date + pd.Timedelta(weeks=6)

# Interest Rates
ffr = pdr.DataReader('FEDFUNDS', 'fred', min_date, max_date)
ffr.columns = ['FederalFundsRate']
prime = pdr.DataReader('PRIME', 'fred', min_date, max_date)
prime.columns = ['PrimeRate']
gs10 = pdr.DataReader('GS10', 'fred', min_date, max_date)
gs10.columns = ['10YrTreasuryRate']

# Inflation
cpi = pdr.DataReader('CPIAUCSL', 'fred', min_date, max_date)
cpi.columns = ['CPI']
core_cpi = pdr.DataReader('CPILFESL', 'fred', min_date, max_date)
core_cpi.columns = ['CoreCPI']

# Economic Output
gdp = pdr.DataReader('GDP', 'fred', min_date, max_date)
gdp.columns = ['GDP']
industrial_production = pdr.DataReader('INDPRO', 'fred', min_date, max_date)
industrial_production.columns = ['IndustrialProduction']

# Labor Market
unemployment_rate = pdr.DataReader('UNRATE', 'fred', min_date, max_date)
unemployment_rate.columns = ['UnemploymentRate']
nonfarm_payrolls = pdr.DataReader('PAYEMS', 'fred', min_date, max_date)
nonfarm_payrolls.columns = ['NonfarmPayments']
avg_hrly_earnings = pdr.DataReader('CES0500000003', 'fred', min_date, max_date)
avg_hrly_earnings.columns = ['AvgHourlyEarnings']

##Corporate Data
corp_profits_after_tax = pdr.DataReader('CP', 'fred', min_date, max_date)
corp_profits_after_tax.columns = ['CorporateProfitsAfterTax']

#Credit Conditions
consumer_credit = pdr.DataReader('CCLACBW027SBOG', 'fred', min_date, max_date)
consumer_credit.columns = ['ConsumerLoans']
bank_lending_data = pdr.DataReader('TOTLL', 'fred', min_date, max_date)
bank_lending_data.columns = ['BankLoansAndLeases']

#Stock Market Data
sp500 = pdr.DataReader('SP500', 'fred', min_date, max_date)
sp500.columns = ['SP500Index']

#Exchange Rates
us_dollar_index = pdr.DataReader('DTWEXBGS', 'fred', min_date, max_date)
us_dollar_index.columns = ['USDIndex']

econ_data = [
    ffr,
    prime,
    gs10,
    cpi,
    core_cpi,
    gdp,
    industrial_production,
    unemployment_rate,
    nonfarm_payrolls,
    avg_hrly_earnings,
    corp_profits_after_tax,
    consumer_credit,
    bank_lending_data,
    sp500,
    us_dollar_index
  ]

econ_df = pd.DataFrame()

for df in econ_data:
  df.index = df.index.tz_localize('America/New_York')
  econ_df = econ_df.merge(df, left_index=True, right_index=True, how='outer')
  econ_df.ffill(inplace=True)

In [322]:
econ_df.tail()

,FederalFundsRate,PrimeRate,10YrTreasuryRate,CPI,CoreCPI,GDP,IndustrialProduction,UnemploymentRate,NonfarmPayments,AvgHourlyEarnings,CorporateProfitsAfterTax,ConsumerLoans,BankLoansAndLeases,SP500Index,USDIndex
DATE,,,,,,,,,,,,,,,
2025-02-05 00:00:00-05:00,4.33,7.5,4.45,319.775,325.475,29723.864,104.2062,4.1,159170.0,35.91,3631.383,1086.0197,12625.7125,6061.48,128.1725
2025-02-06 00:00:00-05:00,4.33,7.5,4.45,319.775,325.475,29723.864,104.2062,4.1,159170.0,35.91,3631.383,1086.0197,12625.7125,6083.57,128.3958
2025-02-07 00:00:00-05:00,4.33,7.5,4.45,319.775,325.475,29723.864,104.2062,4.1,159170.0,35.91,3631.383,1086.0197,12625.7125,6025.99,128.6144
2025-02-10 00:00:00-05:00,4.33,7.5,4.45,319.775,325.475,29723.864,104.2062,4.1,159170.0,35.91,3631.383,1086.0197,12625.7125,6066.44,128.8215
2025-02-11 00:00:00-05:00,4.33,7.5,4.45,319.775,325.475,29723.864,104.2062,4.1,159170.0,35.91,3631.383,1086.0197,12625.7125,6068.50,128.6290


In [323]:
AAPL_q = yf.Ticker('AAPL') \
  .get_balance_sheet(freq='quarterly') \
  .T[[
    'NetDebt',
    'TotalDebt',
    'WorkingCapital',
    'TangibleBookValue',
    'RetainedEarnings',
    'StockholdersEquity',
    'TotalAssets',
    'CashAndCashEquivalents',
    'Inventory',
    'AccountsReceivable',
    'CurrentAssets',
    'CurrentLiabilities',
    'TotalEquityGrossMinorityInterest'
  ]]
AAPL_q = AAPL_q.sort_index()
AAPL_q.insert(0, 'EndDate', AAPL_q.index)
AAPL_q.insert(0, 'StartDate', AAPL_q.index.to_series().shift(1) + pd.Timedelta(days=1))
AAPL_q.insert(0, 'Granularity', 'Quarterly')

AAPL_y = yf.Ticker('AAPL') \
  .get_balance_sheet(freq='yearly') \
  .T[[
    'NetDebt',
    'TotalDebt',
    'WorkingCapital',
    'TangibleBookValue',
    'RetainedEarnings',
    'StockholdersEquity',
    'TotalAssets',
    'CashAndCashEquivalents',
    'Inventory',
    'AccountsReceivable',
    'CurrentAssets',
    'CurrentLiabilities',
    'TotalEquityGrossMinorityInterest'
  ]]
AAPL_y = AAPL_y.sort_index()
AAPL_y.insert(0, 'EndDate', AAPL_y.index)
AAPL_y.insert(0, 'StartDate', AAPL_y.index.to_series().shift(1) + pd.Timedelta(days=1))
AAPL_y.insert(0, 'Granularity', 'Yearly')

AAPL = pd.concat([AAPL_q, AAPL_y])
AAPL.insert(0, 'Ticker', 'AAPL')
AAPL.reset_index(drop=True, inplace=True)
AAPL

,Ticker,Granularity,StartDate,EndDate,NetDebt,TotalDebt,WorkingCapital,TangibleBookValue,RetainedEarnings,StockholdersEquity,TotalAssets,CashAndCashEquivalents,Inventory,AccountsReceivable,CurrentAssets,CurrentLiabilities,TotalEquityGrossMinorityInterest
0,AAPL,Quarterly,NaT,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,Quarterly,2023-10-01,2023-12-31,67280000000.0,108040000000.0,9719000000.0,74100000000.0,8242000000.0,74100000000.0,353514000000.0,40760000000.0,6511000000.0,23194000000.0,143692000000.0,133973000000.0,74100000000.0
2,AAPL,Quarterly,2024-01-01,2024-03-31,71895000000.0,104590000000.0,4594000000.0,74194000000.0,4339000000.0,74194000000.0,337411000000.0,32695000000.0,6232000000.0,21837000000.0,128416000000.0,123822000000.0,74194000000.0
3,AAPL,Quarterly,2024-04-01,2024-06-30,75739000000.0,101304000000.0,-6189000000.0,66708000000.0,-4726000000.0,66708000000.0,331612000000.0,25565000000.0,6165000000.0,22795000000.0,125435000000.0,131624000000.0,66708000000.0
4,AAPL,Quarterly,2024-07-01,2024-09-30,76686000000.0,106629000000.0,-23405000000.0,56950000000.0,-19154000000.0,56950000000.0,364980000000.0,29943000000.0,7286000000.0,33410000000.0,152987000000.0,176392000000.0,56950000000.0
5,AAPL,Quarterly,2024-10-01,2024-12-31,66500000000.0,96799000000.0,-11125000000.0,66758000000.0,-11221000000.0,66758000000.0,344085000000.0,30299000000.0,6911000000.0,29639000000.0,133240000000.0,144365000000.0,66758000000.0
6,AAPL,Yearly,NaT,2020-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AAPL,Yearly,2020-10-01,2021-09-30,89779000000.0,136522000000.0,9355000000.0,63090000000.0,5562000000.0,63090000000.0,351002000000.0,34940000000.0,6580000000.0,26278000000.0,134836000000.0,125481000000.0,63090000000.0
8,AAPL,Yearly,2021-10-01,2022-09-30,96423000000.0,132480000000.0,-18577000000.0,50672000000.0,-3068000000.0,50672000000.0,352755000000.0,23646000000.0,4946000000.0,28184000000.0,135405000000.0,153982000000.0,50672000000.0
9,AAPL,Yearly,2022-10-01,2023-09-30,81123000000.0,111088000000.0,-1742000000.0,62146000000.0,-214000000.0,62146000000.0,352583000000.0,29965000000.0,6331000000.0,29508000000.0,143566000000.0,145308000000.0,62146000000.0


In [324]:
AAPL_all['NetDebt_q'] = pd.NA
AAPL_all['NetDebt_y'] = pd.NA
AAPL_all['TotalDebt_q'] = pd.NA
AAPL_all['TotalDebt_y'] = pd.NA
AAPL_all['WorkingCapital_q'] = pd.NA
AAPL_all['WorkingCapital_y'] = pd.NA
AAPL_all['TangibleBookValue_q'] = pd.NA
AAPL_all['TangibleBookValue_y'] = pd.NA
AAPL_all['RetainedEarnings_q'] = pd.NA
AAPL_all['RetainedEarnings_y'] = pd.NA
AAPL_all['StockholdersEquity_q'] = pd.NA
AAPL_all['StockholdersEquity_y'] = pd.NA
AAPL_all['TotalAssets_q'] = pd.NA
AAPL_all['TotalAssets_y'] = pd.NA
AAPL_all['CashAndCashEquivalents_q'] = pd.NA
AAPL_all['CashAndCashEquivalents_y'] = pd.NA
AAPL_all['Inventory_q'] = pd.NA
AAPL_all['Inventory_y'] = pd.NA
AAPL_all['AccountsReceivable_q'] = pd.NA
AAPL_all['AccountsReceivable_y'] = pd.NA
AAPL_all['CurrentAssets_q'] = pd.NA
AAPL_all['CurrentAssets_y'] = pd.NA
AAPL_all['CurrentLiabilities_q'] = pd.NA
AAPL_all['CurrentLiabilities_y'] = pd.NA
AAPL_all['TotalEquityGrossMinorityInterest_q'] = pd.NA
AAPL_all['TotalEquityGrossMinorityInterest_y'] = pd.NA

In [325]:
granularity_map = {
    'Quarterly': '_q',
    'Yearly': '_y'
}

for row in AAPL.itertuples():
    start = row.StartDate.tz_localize('America/New_York')
    end = row.EndDate.tz_localize('America/New_York')
    granularity = row.Granularity

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'NetDebt{granularity_map[granularity]}'
    ] = row.NetDebt

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'TotalDebt{granularity_map[granularity]}'
    ] = row.TotalDebt

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'WorkingCapital{granularity_map[granularity]}'
    ] = row.WorkingCapital

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'TangibleBookValue{granularity_map[granularity]}'
    ] = row.TangibleBookValue

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'RetainedEarnings{granularity_map[granularity]}'
    ] = row.RetainedEarnings

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'StockholdersEquity{granularity_map[granularity]}'
    ] = row.StockholdersEquity

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'TotalAssets{granularity_map[granularity]}'
    ] = row.TotalAssets

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'CashAndCashEquivalents{granularity_map[granularity]}'
    ] = row.CashAndCashEquivalents

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'Inventory{granularity_map[granularity]}'
    ] = row.Inventory

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'AccountsReceivable{granularity_map[granularity]}'
    ] = row.AccountsReceivable

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'CurrentAssets{granularity_map[granularity]}'
    ] = row.CurrentAssets

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'CurrentLiabilities{granularity_map[granularity]}'
    ] = row.CurrentLiabilities

    AAPL_all.loc[
        (AAPL_all.index >= start) & (AAPL_all.index <= end),
        f'TotalEquityGrossMinorityInterest{granularity_map[granularity]}'
    ] = row.TotalEquityGrossMinorityInterest

In [328]:
AAPL_all = AAPL_all.merge(econ_df, left_index=True, right_index=True, how='left')
AAPL_all.insert(1, 'Sector', yf.Ticker('AAPL').info['sector'])
AAPL_all

ValueError: cannot insert Sector, already exists